In [1]:
import pandas as pd
import numpy as np

class DraftState:
    def __init__(self, rosters, turns, freeagents, playerjm=None):
        self.rosters = rosters
        self.freeagents = freeagents
        self.turns = turns
        self.playerJustMoved = playerjm

class MLBPlayer:
    def __init__(self, name, team, position, points, adp, adj_pts):
        self.name = name
        self.team = team
        self.position = str(position)
        self.points = points
        self.adp = adp
        self.adj_pts = adj_pts
        
    def __repr__(self):
        #return self.name
        return "_".join([self.name, self.position, str(round(self.points)), 'pts'])
    def pop(self, index=-1) :
        return self.data.pop(index)

def GetResult(self, playerjm):
    """ Get the game result from the viewpoint of playerjm.
    """
    if playerjm is None: return 0
    
    pos_wgts = {
        ("IF"): [1, 1],
        ("OF"): [1, 1],
        ("P"): [1],
        ("IF", "OF"):[.7]
        # ("SS"): [1],
        # ('3B'): [1],
        # # ("1B", "3B"): [.6, .4],
        # # ("2B", "SS"): [.6, .4],
        # # ('LF'):[.3],
        # # ('CF'):[.3],
        # ('OF'):[1],
        # ("C", "1B", "2B", "SS", "3B", 'DH'): [1],
        # ("SP"): [1],
        # ("RP"): [1]
    }
    result = 0
    # map the drafted players to the weights
    for p in self.rosters[playerjm]:
        max_wgt, _, max_pos, old_wgts = max(
            ((wgts[0], -len(lineup_pos), lineup_pos, wgts) for lineup_pos, wgts in pos_wgts.items()
                if p.position in lineup_pos),
            default=(0, 0, (), []))
        if max_wgt > 0:
            result += max_wgt * p.points
            old_wgts.pop(0)
            if not old_wgts:
                pos_wgts.pop(max_pos)
                
    # map the remaining weights to the top three free agents
    for pos, wgts in pos_wgts.items():
        result += np.mean([p.points for p in self.freeagents if p.position in pos][:3]) * sum(wgts)
    return result
DraftState.GetResult = GetResult

def GetMoves(self):
    """ Get all possible moves from this state.
    """
    
    pos_max = {"IF": 3, "OF": 3, "P": 1}
    if len(self.turns) == 0: return []
    roster_positions = np.array([p.position for p in self.rosters[self.turns[0]]], dtype=str)
    moves = [pos for pos, max_ in pos_max.items() if np.sum(roster_positions == pos) < max_]
    #moves = list(pos_max.keys())
    return moves
DraftState.GetMoves = GetMoves

def DoMove(self, move):
    """ Update a state by carrying out the given move.
        Must update playerJustMoved.
    """
    player = next(p for p in self.freeagents if p.position == move)
    self.freeagents.remove(player)
    rosterId = self.turns.pop(0)
    self.rosters[rosterId].append(player)
    self.playerJustMoved = rosterId
    
DraftState.DoMove = DoMove

def DoGreedyMove(self, move):
    """ Update a state by carrying out the greediest (most points at the moment) move.
        Must update playerJustMoved.
    """
    # Get the next highest player from the eligible player pool (i.e. you can't draft an 8th IF if you have 7 already)
    player = next(p for p in self.freeagents if p.position == move)
    self.freeagents.remove(player)
    rosterId = self.turns.pop(0)
    self.rosters[rosterId].append(player)
    self.playerJustMoved = rosterId
    
DraftState.DoGreedyMove = DoGreedyMove

def DoAdpMove(self):
    moves = self.GetMoves()
    fa_subset = [p for p in self.freeagents if p.position in moves]
    players = [np.random.normal(loc = p.adp) for p in self.freeagents if p.position in moves]
    ind = players.index(min(players))
    player = fa_subset[ind]
    self.freeagents.remove(player)
    rosterId = self.turns.pop(0)
    self.rosters[rosterId].append(player)
    self.playerJustMoved = rosterId

DraftState.DoAdpMove = DoAdpMove

def DoManualMove(self, player_name):
    """ Update a state by carrying out a specific move.
        Must update playerJustMoved.
    """
    player = [p for p in self.freeagents if p.name == player_name][0]
    self.freeagents.remove(player)
    rosterId = self.turns.pop(0)
    self.rosters[rosterId].append(player)
    self.playerJustMoved = rosterId
    
DraftState.DoManualMove = DoManualMove

def Clone(self):
    """ Create a deep clone of this game state.
    """
    rosters = list(map(lambda r: r[:], self.rosters))
    st = DraftState(rosters, self.turns[:], self.freeagents[:],
            self.playerJustMoved)
    return st
DraftState.Clone = Clone

# Add other strategies:

def greedy_draft(self):
    def result(move):
        st = self.Clone()
        st.DoMove(move)
        return st.GetResult(st.playerJustMoved)
    
    res, move = max((result(m), m) for m in self.GetMoves())
    return move
DraftState.greedy_draft = greedy_draft

def random_draft(self):
    return np.random.choice(self.GetMoves())
DraftState.random_draft = random_draft


# This is a very simple implementation of the UCT Monte Carlo Tree Search algorithm in Python 2.7.
# The function UCT(rootstate, itermax, verbose = False) is towards the bottom of the code.
# It aims to have the clearest and simplest possible code, and for the sake of clarity, the code
# is orders of magnitude less efficient than it could be made, particularly by using a 
# state.GetRandomMove() or state.DoRandomRollout() function.
# 
# Written by Peter Cowling, Ed Powley, Daniel Whitehouse (University of York, UK) September 2012.
# 
# Licence is granted to freely use and distribute for any sensible/legal purpose so long as this comment
# remains in any distributed code.
# 
# For more information about Monte Carlo Tree Search check out our web site at www.mcts.ai
from math import *
import random
class Node:
    """ A node in the game tree. Note wins is always from the viewpoint of playerJustMoved.
        Crashes if state not specified.
    """
    def __init__(self, move = None, parent = None, state = None):
        self.move = move # the move that got us to this node - "None" for the root node
        self.parentNode = parent # "None" for the root node
        self.childNodes = []
        self.wins = 0
        self.visits = 0
        self.untriedMoves = state.GetMoves() # future child nodes
        self.playerJustMoved = state.playerJustMoved # the only part of the state that the Node needs later
        
    def UCTSelectChild(self):
        """ Use the UCB1 formula to select a child node. Often a constant UCTK is applied so we have
            lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits to vary the amount of
            exploration versus exploitation.
        """
        UCTK = 200
        s = sorted(self.childNodes, key = lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits))[-1]
        return s
    
    def AddChild(self, m, s):
        """ Remove m from untriedMoves and add a new child node for this move.
            Return the added child node
        """
        n = Node(move = m, parent = self, state = s)
        self.untriedMoves.remove(m)
        self.childNodes.append(n)
        return n
    
    def Update(self, result):
        """ Update this node - one additional visit and result additional wins. result must be from the viewpoint of playerJustmoved.
        """
        self.visits += 1
        self.wins += result
def UCT(rootstate, itermax, verbose = False):
    """ Conduct a UCT search for itermax iterations starting from rootstate.
        Return the best move from the rootstate.
    """
    rootnode = Node(state = rootstate)
    for i in range(itermax):
        node = rootnode
        state = rootstate.Clone()
        # Select
        while node.untriedMoves == [] and node.childNodes != []: # node is fully expanded and non-terminal
            node = node.UCTSelectChild()
            state.DoMove(node.move)
        # Expand
        if node.untriedMoves != []: # if we can expand (i.e. state/node is non-terminal)
            m = random.choice(node.untriedMoves) 
            state.DoMove(m)
            node = node.AddChild(m,state) # add child and descend tree
        # Rollout - this can often be made orders of magnitude quicker using a state.GetRandomMove() function
        while state.GetMoves() != []: # while state is non-terminal
            state.DoMove(random.choice(state.GetMoves()))
        # Backpropagate
        while node != None: # backpropagate from the expanded node and work back to the root node
            node.Update(state.GetResult(node.playerJustMoved)) # state is terminal. Update node with result from POV of node.playerJustMoved
            node = node.parentNode
    return sorted(rootnode.childNodes, key = lambda c: c.visits)[-1].move # return the move that was most visited

In [10]:
def show_results(self):
    tm1 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[0])])
    tm2 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[1])])
    tm3 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[2])])
    # tm4 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[3])])
    # tm5 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[4])])
    # tm6 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[5])])
    # tm7 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[6])])
    # tm8 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[7])])
    # tm9 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[8])])
    # tm10 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[9])])
    # tm11 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[10])])
    # tm12 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[11])])
    # results = pd.concat([tm1, tm2, tm3, tm4, tm5, tm6, tm7, tm8, tm9, tm10, tm11, tm12], axis = 1)
    # results.columns = ['tm1', 'tm2', 'tm3', 'tm4', 'tm5', 'tm6', 'tm7', 'tm8', 'tm9', 'tm10', 'tm11', 'tm12']

    results = pd.concat([tm1, tm2, tm3], axis = 1)
    # results.columns = ['tm1', 'tm2', 'tm3']
    return results

In [41]:
mlb_players = pd.read_csv("~/Documents/GitHub/fantasy_baseball_draft_bot/data/underdog_daily_data/cleaned/3_31.csv")

freeagents = [MLBPlayer(*p) for p in mlb_players.itertuples(index=False, name=None)]

num_competitors = 3
rosters = [[] for _ in range(num_competitors)] # empty rosters to start with

num_rounds = 6
turns = []
# generate turns by snake order
for i in range(num_rounds):
    turns += reversed(range(num_competitors)) if i % 2 else range(num_competitors)
    
state = DraftState(rosters, turns, freeagents)
iterations = 1000

# Do Manual Move (Other Players)

In [ ]:
temp = state.Clone()
state.DoManualMove('')
show_results(state)

# Do AI Move

In [ ]:
temp = state.Clone()
move = UCT(state, iterations)
state.DoMove(move)
show_results(state)

In [45]:
UCT(state, iterations)

'P'